# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 13 2023 8:30AM,254964,10,MSP217521,"Methapharm, Inc.",Released
1,Jan 13 2023 8:30AM,254964,10,MSP217522,"Methapharm, Inc.",Released
2,Jan 13 2023 8:30AM,254964,10,MSP217523,"Methapharm, Inc.",Released
3,Jan 13 2023 8:30AM,254967,10,0086296197,ISDIN Corporation,Released
4,Jan 13 2023 8:30AM,254967,10,0086296202,ISDIN Corporation,Released
5,Jan 13 2023 8:30AM,254963,10,0086296211,ISDIN Corporation,Released
6,Jan 13 2023 8:30AM,254959,10,SOIVA0000574,Ivaoes Animal Health,Released
7,Jan 13 2023 8:30AM,254956,10,8838520-BO,Eywa Pharma Inc.,Released
8,Jan 13 2023 8:30AM,254956,10,8868344-BO,Eywa Pharma Inc.,Released
9,Jan 13 2023 8:30AM,254965,10,0086296218,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,254963,Released,1
28,254964,Released,3
29,254965,Released,1
30,254966,Released,9
31,254967,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254963,NaN,NaN,1.0
254964,NaN,NaN,3.0
254965,NaN,NaN,1.0
254966,NaN,NaN,9.0
254967,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254864,35.0,7.0,4.0
254867,8.0,5.0,4.0
254874,0.0,0.0,1.0
254899,0.0,0.0,1.0
254905,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254864,35,7,4
254867,8,5,4
254874,0,0,1
254899,0,0,1
254905,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254864,35,7,4
1,254867,8,5,4
2,254874,0,0,1
3,254899,0,0,1
4,254905,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254864,35,7,4
1,254867,8,5,4
2,254874,,,1
3,254899,,,1
4,254905,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 13 2023 8:30AM,254964,10,"Methapharm, Inc."
3,Jan 13 2023 8:30AM,254967,10,ISDIN Corporation
5,Jan 13 2023 8:30AM,254963,10,ISDIN Corporation
6,Jan 13 2023 8:30AM,254959,10,Ivaoes Animal Health
7,Jan 13 2023 8:30AM,254956,10,Eywa Pharma Inc.
9,Jan 13 2023 8:30AM,254965,10,ISDIN Corporation
11,Jan 12 2023 4:02PM,254966,10,ISDIN Corporation
20,Jan 12 2023 3:42PM,254961,12,"SD Head USA, LLC"
21,Jan 12 2023 3:41PM,254960,22,"NBTY Global, Inc."
22,Jan 12 2023 3:12PM,254957,20,"Exact-Rx, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 13 2023 8:30AM,254964,10,"Methapharm, Inc.",,,3
1,Jan 13 2023 8:30AM,254967,10,ISDIN Corporation,,,2
2,Jan 13 2023 8:30AM,254963,10,ISDIN Corporation,,,1
3,Jan 13 2023 8:30AM,254959,10,Ivaoes Animal Health,,,1
4,Jan 13 2023 8:30AM,254956,10,Eywa Pharma Inc.,,,3
5,Jan 13 2023 8:30AM,254965,10,ISDIN Corporation,,,1
6,Jan 12 2023 4:02PM,254966,10,ISDIN Corporation,,,9
7,Jan 12 2023 3:42PM,254961,12,"SD Head USA, LLC",,,1
8,Jan 12 2023 3:41PM,254960,22,"NBTY Global, Inc.",,,1
9,Jan 12 2023 3:12PM,254957,20,"Exact-Rx, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 13 2023 8:30AM,254964,10,"Methapharm, Inc.",3,,
1,Jan 13 2023 8:30AM,254967,10,ISDIN Corporation,2,,
2,Jan 13 2023 8:30AM,254963,10,ISDIN Corporation,1,,
3,Jan 13 2023 8:30AM,254959,10,Ivaoes Animal Health,1,,
4,Jan 13 2023 8:30AM,254956,10,Eywa Pharma Inc.,3,,
5,Jan 13 2023 8:30AM,254965,10,ISDIN Corporation,1,,
6,Jan 12 2023 4:02PM,254966,10,ISDIN Corporation,9,,
7,Jan 12 2023 3:42PM,254961,12,"SD Head USA, LLC",1,,
8,Jan 12 2023 3:41PM,254960,22,"NBTY Global, Inc.",1,,
9,Jan 12 2023 3:12PM,254957,20,"Exact-Rx, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 13 2023 8:30AM,254964,10,"Methapharm, Inc.",3,,
1,Jan 13 2023 8:30AM,254967,10,ISDIN Corporation,2,,
2,Jan 13 2023 8:30AM,254963,10,ISDIN Corporation,1,,
3,Jan 13 2023 8:30AM,254959,10,Ivaoes Animal Health,1,,
4,Jan 13 2023 8:30AM,254956,10,Eywa Pharma Inc.,3,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 13 2023 8:30AM,254964,10,"Methapharm, Inc.",3.0,NaN,NaN
1,Jan 13 2023 8:30AM,254967,10,ISDIN Corporation,2.0,NaN,NaN
2,Jan 13 2023 8:30AM,254963,10,ISDIN Corporation,1.0,NaN,NaN
3,Jan 13 2023 8:30AM,254959,10,Ivaoes Animal Health,1.0,NaN,NaN
4,Jan 13 2023 8:30AM,254956,10,Eywa Pharma Inc.,3.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 13 2023 8:30AM,254964,10,"Methapharm, Inc.",3.0,0.0,0.0
1,Jan 13 2023 8:30AM,254967,10,ISDIN Corporation,2.0,0.0,0.0
2,Jan 13 2023 8:30AM,254963,10,ISDIN Corporation,1.0,0.0,0.0
3,Jan 13 2023 8:30AM,254959,10,Ivaoes Animal Health,1.0,0.0,0.0
4,Jan 13 2023 8:30AM,254956,10,Eywa Pharma Inc.,3.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3314341,54.0,17.0,0.0
12,254961,1.0,0.0,0.0
16,254874,1.0,0.0,0.0
17,254899,1.0,0.0,0.0
19,1274488,10.0,14.0,43.0
20,254957,1.0,0.0,0.0
21,509840,2.0,0.0,0.0
22,254960,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3314341,54.0,17.0,0.0
1,12,254961,1.0,0.0,0.0
2,16,254874,1.0,0.0,0.0
3,17,254899,1.0,0.0,0.0
4,19,1274488,10.0,14.0,43.0
5,20,254957,1.0,0.0,0.0
6,21,509840,2.0,0.0,0.0
7,22,254960,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,54.0,17.0,0.0
1,12,1.0,0.0,0.0
2,16,1.0,0.0,0.0
3,17,1.0,0.0,0.0
4,19,10.0,14.0,43.0
5,20,1.0,0.0,0.0
6,21,2.0,0.0,0.0
7,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,54.0
1,12,Released,1.0
2,16,Released,1.0
3,17,Released,1.0
4,19,Released,10.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,17,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,0.0,0.0,43.0,0.0,0.0,0.0
Executing,17.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0
Released,54.0,1.0,1.0,1.0,10.0,1.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,17,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,43.0,0.0,0.0,0.0
1,Executing,17.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0
2,Released,54.0,1.0,1.0,1.0,10.0,1.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,17,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,43.0,0.0,0.0,0.0
1,Executing,17.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0
2,Released,54.0,1.0,1.0,1.0,10.0,1.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()